In [1]:
from tqdm import tqdm_notebook as tqdm
import nltk
import numpy as np
import pandas as pd
import json
import re

# put existing gzt and files into pd df

In [6]:
with open('data/acrolinx_gzt/lf.json') as lfjson:
    lf = json.load(lfjson)

In [10]:
with open('data/acrolinx_gzt/conv-words.json') as cvjson:
    form_with_sugg = json.load(cvjson)

In [17]:
unclean_gzt = []

with open('data/acrolinx_gzt/archaicWords.gzt') as file:
    unclean_gzt.extend(file.readlines())

with open('data/acrolinx_gzt/countFormalPhrases.gzt') as file:
    unclean_gzt.extend(file.readlines())

with open('data/acrolinx_gzt/countLatinExpressions.gzt') as file:
    unclean_gzt.extend(file.readlines())

In [69]:
gzt = {}

# things i noticed and don't want
exceptions = ['use either', '(']

for item in unclean_gzt:
    if item[0] == '@' or item[0] == '#':
        continue
    item = item.strip()
    if len(item) < 1:
        continue
    trigger = False
    for term in exceptions:
        if term in item:
            trigger = True
    if trigger:
        continue
    item = re.sub('\[', '', item)
    item = re.sub('\]', '', item)
    item = re.sub('\n', '', item)
    item = re.sub(';', '', item)
    if '-->' in item:
        pair = [part.strip() for part in item.split('-->')]
        if ',' in pair[0]:
            form_words = [part.strip() for part in pair[0].split(',')]
            for word in form_words:
                gzt[word] = [part.strip() for part in pair[1].split(',')]
        else:
            gzt[pair[0]] = [part.strip() for part in pair[1].split(',')]
    else:
        gzt[item] = np.nan

In [71]:
len(gzt)

597

In [72]:
formal = []
informal = []

for word in gzt:
    formal.append(word)
    informal.append(gzt[word])

words = pd.DataFrame()
words['formal'] = formal
words['suggestions'] = informal
words.head()

,formal,suggestions
0,purchase,[buy]
1,caveat,[warning]
2,asserted,NaN
3,thenceforward,NaN
4,Pardon me,[Sorry]


In [73]:
words.to_pickle('data/acrolinx_gzt/initial_words.pkl')

# next: extrapolate to the other words using embeddings

In [2]:
words_df = pd.read_pickle('data/acrolinx_gzt/initial_words.pkl')

In [3]:
words_df.head()

,formal,suggestions
0,purchase,[buy]
1,caveat,[warning]
2,asserted,NaN
3,thenceforward,NaN
4,Pardon me,[Sorry]


In [ ]:
# start with the 1 token words

# with the phrases, go on to use the NMT model developed in 07